In [26]:
using Base.Threads
using LinearAlgebra
using ExtendableSparse
# using Surrogates

# Define a function to make a copy of tuples
_copy(t::Tuple) = t
_copy(t) = copy(t)

# Define an abstract type for Surrogate
abstract type AbstractSurrogate end

# Define the RadialBasis type
mutable struct RadialBasis{F, Q, X, Y, L, U, C, S, D} <: AbstractSurrogate
    phi::F
    dim_poly::Q
    x::X
    y::Y
    lb::L
    ub::U
    coeff::C
    scale_factor::S
    sparse::D
end

# Define the RadialFunction type
mutable struct RadialFunction{Q, P}
    q::Q # degree of polynomial
    phi::P
end

# Define radial basis functions
linearRadial() = RadialFunction(0, z -> norm(z))
cubicRadial() = RadialFunction(1, z -> norm(z)^3)
multiquadricRadial(c = 1.0) = RadialFunction(1, z -> sqrt((c * norm(z))^2 + 1))
thinplateRadial() = RadialFunction(2, z -> begin
    result = norm(z)^2 * log(norm(z))
    ifelse(iszero(z), zero(result), result)
end)

# Define RadialBasis constructor
function RadialBasis(x, y, lb, ub; rad::RadialFunction = linearRadial(),
                      scale_factor::Real = 0.5, sparse = false)
    q = rad.q
    phi = rad.phi
    coeff = _calc_coeffs(x, y, lb, ub, phi, q, scale_factor, sparse)
    return RadialBasis(phi, q, x, y, lb, ub, coeff, scale_factor, sparse)
end

# Calculate coefficients for RadialBasis
function _calc_coeffs(x, y, lb, ub, phi, q, scale_factor, sparse)
    nd = length(first(x))
    num_poly_terms = binomial(q + nd, q)
    D = _construct_rbf_interp_matrix(x, first(x), lb, ub, phi, q, scale_factor, sparse)
    Y = _construct_rbf_y_matrix(y, first(y), length(y) + num_poly_terms)
    if (typeof(y) == Vector{Float64}) #single output case
        coeff = _copy(transpose(D \ y))
    else
        coeff = _copy(transpose(D \ Y[1:size(D)[1], :])) #if y is multi output;
    end
    return coeff
end

# Construct the interpolation matrix
function _construct_rbf_interp_matrix(x, x_el, lb, ub, phi, q, scale_factor, sparse)
    n = length(x)
    nd = length(x_el)
    if sparse
        D = ExtendableSparseMatrix{eltype(x_el), Int}(n, n)
    else
        D = zeros(eltype(x_el), n, n)
    end
    @inbounds for i in 1:n
        for j in i:n
            D[i, j] = phi((x[i] .- x[j]) ./ scale_factor)
        end
    end
    D_sym = Symmetric(D, :U)
    return D_sym
end

# Construct the y matrix
function _construct_rbf_y_matrix(y, y_el, m)
    [i <= length(y) ? y[i] : zero(y_el) for i in 1:m]
end

# Define the multivariate polynomial basis function
function multivar_poly_basis(x, ix, d, n)
    if n == 0
        return one(eltype(x))
    else
        prod(a^d for (a, d) in zip(x, _make_combination(n, d, ix)))
    end
end

# Define the evaluation function for RadialBasis
function (rad::RadialBasis)(val)
    _check_dimension(rad, val)
    approx = _approx_rbf(val, rad)
    return _match_container(approx, first(rad.y))
end

# Approximate the RadialBasis function
function _approx_rbf(val, rad::RadialBasis)
    n = length(rad.x)
    approx = zero(rad.coeff[:, 1])
    for i in 1:n
        approx += rad.coeff[:, i] * rad.phi((val .- rad.x[i]) / rad.scale_factor)
    end
    return approx
end

# Function to add new samples to RadialBasis
function add_point!(rad::RadialBasis, new_x, new_y)
    if (length(new_x) == 1 && length(new_x[1]) == 1) ||
       (length(new_x) > 1 && length(new_x[1]) == 1 && length(rad.lb) > 1)
        push!(rad.x, new_x)
        push!(rad.y, new_y)
    else
        append!(rad.x, new_x)
        append!(rad.y, new_y)
    end
    rad.coeff = _calc_coeffs(rad.x, rad.y, rad.lb, rad.ub, rad.phi, rad.dim_poly,
                              rad.scale_factor, rad.sparse)
    nothing
end

# Define a simple Buffer type
mutable struct Buffer{T}
    value::T
end

# Define a function to make an approximation buffer
function _make_approx(val, rad::RadialBasis)
    l = size(rad.coeff, 1)
    return Buffer(zeros(eltype(val), l))
end

# Define a function to add temporary values to the approximation
function _add_tmp_to_approx!(approx::Buffer, i, tmp, rad::RadialBasis; f = identity)
    @inbounds @simd ivdep for j in 1:size(rad.coeff, 1)
        approx.value[j] += rad.coeff[j, i] * f(tmp)
    end
end

# Define a function to return a copy of the value stored in a Buffer
_ret_copy(v::Buffer) = copy(v.value)

# Function to approximate RadialBasis in a multi-threaded manner
function _approx_rbf(val, rad::RadialBasis)
    n = length(rad.x)
    approx = _make_approx(val, rad)
    Threads.@threads for i in 1:n
        tmp = zero(eltype(val))
        @inbounds @simd ivdep for j in 1:length(val)
            tmp += ((val[j] - rad.x[i][j]) / rad.scale_factor)^2
        end
        tmp = sqrt(tmp)
        _add_tmp_to_approx!(approx, i, tmp, rad)
    end
    return _ret_copy(approx)
end

# Test function for comparing multi-threaded and single-threaded results
function test_approx_rbf(val, x, coeff, scale_factor, phi)
    # Non-multi-threaded version
    result_single_threaded = _approx_rbf(val, RadialBasis(x, [], [], [], [], [], coeff, scale_factor, false))

    # Multi-threaded version
    result_multi_threaded = _approx_rbf_threaded(val, RadialBasis(x, [], [], [], [], [], coeff, scale_factor, false))

    # Compare results
    println("Single-threaded result: ", result_single_threaded)
    println("Multi-threaded result: ", result_multi_threaded)
    
    # Check if results are approximately equal
    if isapprox(result_single_threaded, result_multi_threaded)
        println("Results are approximately equal.")
    else
        println("Results are different.")
    end
end
    
# Run the test with sample data
val = [1.0, 2.0, 3.0]  # Example value for testing
x = [[0.0, 0.0], [1.0, 1.0], [2.0, 2.0]]  # Example radial basis center points
coeff = [1.0, 2.0, 3.0]  # Example coefficients
scale_factor = 1.0  # Example scale factor
phi = linearRadial().phi  # Example radial basis function

test_approx_rbf(val, x, coeff, scale_factor, phi)
    


In [23]:
using BenchmarkTools

# Define your input data
val = [1.0, 2.0, 3.0]  # Example value for testing
x = [[0.0, 0.0], [1.0, 1.0], [2.0, 2.0]]  # Example radial basis center points
coeff = [1.0, 2.0, 3.0]  # Example coefficients
scale_factor = 1.0  # Example scale factor
phi = linearRadial().phi  # Example radial basis function

# Benchmark the single-threaded code
single_threaded_time = @belapsed test_approx_rbf($val, $x, $coeff, $scale_factor, $phi)

# Benchmark the multi-threaded code
multi_threaded_time = @belapsed _approx_rbf_threaded($val, RadialBasis($x, [], [], [], [], [], $coeff, $scale_factor, false))



3.9875e-6

In [24]:
single_threaded_time
# println("Multi-threaded time: ", multi_threaded_time)


0.0009883

In [25]:
multi_threaded_time

3.9875e-6